In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 

# Aufgabe 14

## a)
Logarithmieren hilft bestimmt. 
Wenn Größenordnung stark verschieden, ist es vielleicht sinnvoller ein kleineres $k$ zu wählen um vorzubeugen, dass in den großen Sprüngen dazwischen keine Daten liegen und vermeintlicher Noise als richtiger Wert gewertet wird. 

## b) 

Das Lernen wird durch simples Abspeichern der Trainingsdaten implementiert, insofern ist das lernen sehr ineffizient "lazy". Vermutlich wird es mit $n^k$ laufen. In der Anwendung ist das Verfahren dann aber effizient, da es lediglich mit den vorhandenen Daten abgleichen muss, also vermutlich sogar mit log(n). 

## c) 


In [ ]:
def dist(sample, training): #evtl mit for-loop, wenn so nicht möglich
    distance = np.sqrt((sample[0]-training[,0])**2 + (sample[1]-training[,1])**2) #euklidische Norm sqrt(x^2+y^2)
    return distance

def check(knearest, traininglabel):
    sig, back = 0, 0
    for i in len(knearest):
        if knearest[i] in trainingslabel[0]:
            sig += 1
        elif knearest[i] in trainingslabel[1]:
            back += 1 
        else: 
            print("Some mistake occured.")
    if sig >= back: return 0 
    else: return 1
            
    
def kNN(sample, training, traininglabeled, k):
    distance = dist(sample, training)
    sortdist = np.argsort(distance)
    knearest = sortdist[:k]
    pos = check(knearest, traininglabeled)
    if pos == 0: print("Sample is Signal.")
    else : print("Sample is Background.")
    
    

## d)

In [ ]:
neutrino = pd.read_hdf5("NeutrinoMC.hdf5")
AnzahlHits, x, y = ... 
k = 10 
training = np.array([AnzahlHits[:5000], x[:5000], y[:5000]]) 
test = np.array([AnzahlHits[5000:], x[5000:], y[5000:]]) 

training = np.([x,y])
tra_sig, tra_back = 
traininglabeled = np.array([tra_sig], [tra_back])

for i in range(len(test)):
    kNNlist[]
    kNNlist[i] = kNN(test[i], training, traininglabeled, 10)
    
reinheit = 
effizienz = 
signifikanz = 

print(f"Reinheit: {reinheit*100}:.2f\nEffizienz: {effizienz*100}:.2f\n Signifikanz: {signifikanz*100}:.2f")    

## e) 

In [ ]:
neutrino = pd.read_hdf5("NeutrinoMC.hdf5")
AnzahlHits, x, y = ... 
k = 10 
training = np.array([np.log10(AnzahlHits[:5000]), x[:5000], y[:5000]]) 
test = np.array([AnzahlHits[5000:], x[5000:], y[5000:]]) 

training = np.([x,y])
tra_sig, tra_back = 
traininglabeled = np.array([tra_sig], [tra_back])

for i in range(len(test)):
    kNNlist[]
    kNNlist[i] = kNN(test[i], training, traininglabeled, 10)
    
reinheit = 
effizienz = 
signifikanz = 

print(f"Reinheit: {reinheit*100}:.2f\nEffizienz: {effizienz*100}:.2f\n Signifikanz: {signifikanz*100}:.2f")    

## f) 

In [ ]:
neutrino = pd.read_hdf5("NeutrinoMC.hdf5")
AnzahlHits, x, y = ... 
k = 20 
training = np.array([np.log10(AnzahlHits[:5000]), x[:5000], y[:5000]]) 
test = np.array([AnzahlHits[5000:], x[5000:], y[5000:]]) 

training = np.([x,y])
tra_sig, tra_back = 
traininglabeled = np.array([tra_sig], [tra_back])

for i in range(len(test)):
    kNNlist[]
    kNNlist[i] = kNN(test[i], training, traininglabeled, 10)
    
reinheit = 
effizienz = 
signifikanz = 

print(f"Reinheit: {reinheit*100}:.2f\nEffizienz: {effizienz*100}:.2f\n Signifikanz: {signifikanz*100}:.2f") 

# 15 

## a)

In [ ]:
p0 = pd.read_hdf5("./zwei_populationen.h5", key="P_0_10000")
p1 = pd.read_hdf5("./zwei_populationen.h5", key="P_1")

plt.figure()
plt.scatter(p0)
plt.scatter(p1)
x = np.linspace(p0.min(), p0.max(), 10000)
y1 = 0*x
y2 = -(3/4)*x
y3 = -(5/4)*x
g1 = np.array([x,y1])
g2 = np.array([x,y2])
g3 = np.array([x,y3])
plt.plot(x, g1)
plt.plot(x, g2)
plt.plot(x, g3)

## b) 

In [ ]:
def proj(p):
    return p@g1/(g1@g1), p@g2/(g2@g2), p@g3/(g3@g3)

p0_1, p0_2, p0_3 = proj(p0)
p1_1, p1_2, p1_3 = proj(p1)  

np.hist(p0_1)
np.hist(p0_2)
np.hist(p0_3)

np.hist(p1_1)
np.hist(p1_2)
np.hist(p1_3)

## c) 

In [ ]:
copy paste aus letztem mal